In [9]:
import os

In [15]:
# check the project working directory
%pwd

'c:\\Text-Summarizer-NLP\\research'

In [16]:
os.chdir('../')

In [17]:
%pwd

'c:\\Text-Summarizer-NLP'

In [18]:
from dataclasses import dataclass           # This is a decorator from the dataclasses module in Python that is used to automatically generate special methods in classes, such as __init__, __repr__, and __eq__.
from pathlib import Path                    # This is a class from the pathlib module which provides a convenient way to work with file system paths. It allows you to handle file paths in a platform-independent manner.

@dataclass(frozen=True)                     # This decorator converts the DataIngestionConfig class into a dataclass. The frozen=True parameter makes instances of this class immutable, meaning that once an instance is created, you cannot modify its attributes.
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [113]:
from src.Text_Summarizer.constants import *                                    # import the file paths stored in 'constants' folder in src
from src.Text_Summarizer.utils.common import read_yaml, create_directories       # import those util functions 


In [114]:
class ConfigurationManager:
    def __init__(self, config_filepath = "config/config.yaml", params_filepath = "params.yaml"):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])                   # for 'artifacts_root' in config.yaml file

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:            # as defined in config.yaml, 'data_ingestion'
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [115]:
import pathlib  # Import the pathlib module

class ConfigurationManager:
    def __init__(self, config_filepath: pathlib.Path = pathlib.Path("config/config.yaml"), 
                 params_filepath: pathlib.Path = pathlib.Path("params.yaml")):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [116]:
import os
import urllib.request as request
import zipfile
from src.Text_Summarizer.logging import logger                # import custom logger
from src.Text_Summarizer.utils.common import get_size         # from utils folder import 'get_size' function

In [117]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [118]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-17 18:54:56,773: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-17 18:54:56,777: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-17 18:54:56,781: INFO: common: created directory at: artifacts]
[2024-07-17 18:54:56,782: INFO: common: created directory at: artifacts/data_ingestion]


[2024-07-17 18:55:02,726: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 34D6:9D28E:41C0A9:55FE61:6697C62B
Accept-Ranges: bytes
Date: Wed, 17 Jul 2024 13:25:00 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4731-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1721222700.615926,VS0,VE705
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 5412e5715b304526fa569a721e97ad3339e7fad3
Expires: Wed, 17 Jul 2024 13:30:00 GMT
Source-Age: 0

]
